In [ ]:
# 라벨링 데이터 json으로 만들기 최종?본

import pandas as pd
import re
import json
import os

df = pd.read_csv("musinsa_image_crawling 2.csv")

# 열 인덱스 공백 제거
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()

# '품번' 열을 행 인덱스로 설정
df.set_index('품번', inplace=True)

# '시즌' 열 라벨링
def clean_season(value):
    if pd.isna(value) or 'ALL' in value:
        return 'ALL'
    elif 'SS' in value:
        return 'SS'
    elif 'FW' in value:
        return 'FW'
    else:
        return value

df['시즌'] = df['시즌'].apply(clean_season)

# '패턴' 열 라벨링
def determine_pattern(product_name):
    if any(keyword in product_name.lower() for keyword in ['체크', 'check']):
        return '체크'
    elif any(keyword in product_name.lower() for keyword in ['스트라이프', 'stripe']):
        return '스트라이프'
    else:
        return '단색'
       
df['패턴'] = df['상품명'].apply(determine_pattern)


# '색상' 열 라벨링
color_mapping = {
    '블랙': 'BLACK', '검정': 'BLACK', 'black': 'BLACK', 'Black': 'BLACK',
    '블루': 'BLUE', 'blue': 'BLUE', 'Blue': 'BLUE', '하늘': 'BLUE', 'sky': 'BLUE', '연청': 'BLUE', '스카이': 'BLUE',
    '인디고': 'BLUE', 'indigo': 'BLUE', 'MARINA': 'BLUE', 'DENIM': 'BLUE',
    '챠콜': 'CHARCOAL', '차콜': 'CHARCOAL', 'charcoal': 'CHARCOAL', 'Charcoal': 'CHARCOAL',
    '그레이': 'GRAY', 'gray': 'GRAY', 'Gray': 'GRAY', 'GREY': 'GRAY', '회색': 'GRAY', '메란지': 'GRAY', 
    '멜란지': 'GRAY', 'MELANGE': 'GRAY',
    '네이비': 'NAVY', 'navy': 'NAVY', 'Navy': 'NAVY', '남색': 'NAVY',
    '화이트': 'WHITE', 'white': 'WHITE', 'White': 'WHITE', '에그렛': 'WHITE', 
    '레드': 'RED', 'red': 'RED', 'Red': 'RED',
    '핑크': 'PINK', 'pink': 'PINK', 'Pink': 'PINK',
    '옐로우': 'YELLOW', 'yellow': 'YELLOW', 'Yellow': 'YELLOW', 'LEMON': 'YELLOW', '레몬': 'YELLOW',
    '그린': 'GREEN', 'green': 'GREEN', 'Green': 'GREEN', '아보카도': 'GREEN',
    '베이지': 'BEIGE', 'beige': 'BEIGE', 'Beige': 'BEIGE', '샌드': 'BEIGE', '에크루': 'BEIGE', 'SAHARA': 'BEIGE',
    '머쉬룸': 'BEIGE', '오트밀': 'BEIGE', 'oatmeal': 'BEIGE', 'Oatmeal': 'BEIGE',
    '브라운': 'BROWN', 'brown': 'BROWN', 'Brown': 'BROWN', 'MOCHA': 'BROWN', '모카': 'BROWN', '다크어스': 'BROWN',
    'BRICK': 'BROWN', 'CINNAMON': 'BROWN', '마룬': 'BROWN', 'maroon': 'BROWN', 'Maroon': 'BROWN', 'MAROON': 'BROWN',
    '보라': 'PURPLE', 'purple': 'PURPLE', 'Purple': 'PURPLE', '퍼플': 'PURPLE', '라벤더': 'PURPLE', '바이올렛': 'PURPLE',
    '오렌지': 'ORANGE', 'orange': 'ORANGE', 'Orange': 'ORANGE', '탠저린': 'ORANGE', '코랄': 'ORANGE', 'CORAL': 'ORANGE',
    '민트': 'MINT', 'mint': 'MINT', 'Mint': 'MINT',
    '크림': 'CREAM', 'cream': 'CREAM', 'Cream': 'CREAM',
    '아이보리': 'IVORY', 'ivory': 'IVORY', 'Ivory': 'IVORY', '네츄럴': 'IVORY',
    '올리브': 'OLIVE', 'olive': 'OLIVE', 'Olive': 'OLIVE',
    '버건디': 'BURGUNDY', 'burgundy': 'BURGUNDY', 'Burgundy': 'BURGUNDY',
    '와인': 'WINE', 'wine': 'WINE', 'Wine': 'WINE',
    '카키': 'KHAKI', 'khaki': 'KHAKI', 'Khaki': 'KHAKI',
    '메탈': 'METAL', '골드': 'GOLD',
    '스트라이프': 'STRIPE', 'stripe': 'STRIPE', 'Stripe': 'STRIPE'
}


# 정규 표현식 패턴 생성
pattern = '|'.join(color_mapping.keys())

# 색상 추출 및 변환 함수 정의
def extract_and_convert_color(product_name):
    match = re.search(pattern, product_name, re.IGNORECASE)
    if match:
        try:
            return color_mapping[match.group().lower()]  # 소문자로 변환하여 검색
        except KeyError:
            return 'MULTICOLOR'  # 만약 키가 없으면 MULTICOLOR 반환
    else:
        return 'MULTICOLOR'  # 색상이 명시되지 않은 경우

# 새로운 열에 색상 정보 추가
df['추출된 색상'] = df['상품명'].apply(extract_and_convert_color)

# CSV 파일 읽기
file_path = 'musinsa_image_crawling 2.csv'  # 파일 경로를 실제 파일 위치로 설정하세요
df = pd.read_csv(file_path)

# JSON 데이터 구조 생성
json_data = {"images": []}

for _, row in df.iterrows():
    image_data = {
        "image_id": row.get('상품URL', '').split('/')[-1] + '.jpg',  # '상품URL' 열에서 이미지 파일명 추출
        "품번": row['품번'],
        "카테고리": row['카테고리'],
        "브랜드": row['브랜드'],
        "성별": row['성별'],
        "시즌": row['시즌'],
        "클래스": row['품번'],  # 품번을 그대로 클래스 이름으로 사용
        "이미지 URL": row['상품URL'],
        "바운딩 박스": {
            "x_min": None,
            "y_min": None,
            "x_max": None,
            "y_max": None
        }
    }
    json_data["images"].append(image_data)

# JSON 파일 저장 경로 설정
desired_folder = 'C:/Users/1102w/Downloads'  # 원하는 경로로 수정
json_file_path = os.path.join(desired_folder, '데이터라벨링.json')

# 디렉터리가 존재하지 않으면 생성
if not os.path.exists(desired_folder):
    os.makedirs(desired_folder)

# JSON 파일로 저장
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"JSON 파일이 {json_file_path}에 성공적으로 저장되었습니다.")